In [1]:
import pandas as pd
import numpy as np
# import trading_strategy_utils as tsu
import update_dataframe as ud
import grid_search as gs

In [2]:
# Load the doge data
doge_df = pd.read_csv('data/DOGE.csv', parse_dates=['ts_hour'])

# Select March in 2020
doge_df_march_22 = doge_df[(doge_df['ts_hour'].dt.month == 3) & (doge_df['ts_hour'].dt.year == 2022)]
doge_df_march_22 = doge_df_march_22[['ts_hour', 'pnl']]

doge_df_march_22.head()
# doge_df.head()

,ts_hour,pnl
18933,2022-03-01 00:00:00,-0.002309
18934,2022-03-01 01:00:00,0.005154
18935,2022-03-01 02:00:00,0.000265
18936,2022-03-01 03:00:00,0.001963
18937,2022-03-01 04:00:00,0.000869


In [3]:
doge_df_march_22.tail()

,ts_hour,pnl
19672,2022-03-31 19:00:00,0.003776
19673,2022-03-31 20:00:00,-0.003158
19674,2022-03-31 21:00:00,-0.001191
19675,2022-03-31 22:00:00,0.005807
19676,2022-03-31 23:00:00,0.003243


### Grid Search for EMA(Short & Long) params

In [4]:
# Define the range of parameters for grid search
short_window_range = range(4, 7, 1)
long_window_range = range(11, 15, 1)


# Perform grid search
grid_search_march_22 = gs.grid_search_2_params(doge_df_march_22, short_window_range, long_window_range)
grid_search_march_22_sorted = grid_search_march_22.sort_values('total_weighted_pnl', ascending=False)

short_window: 4, long_window: 11, total_weighted_pnl: 0.04687271463361433
short_window: 4, long_window: 12, total_weighted_pnl: 0.049469074837304526
short_window: 4, long_window: 13, total_weighted_pnl: 0.044479572913463696
short_window: 4, long_window: 14, total_weighted_pnl: 0.04914493958863334
short_window: 5, long_window: 11, total_weighted_pnl: 0.04458462318103834
short_window: 5, long_window: 12, total_weighted_pnl: 0.043176289501419676
short_window: 5, long_window: 13, total_weighted_pnl: 0.044856715548582976
short_window: 5, long_window: 14, total_weighted_pnl: 0.046792634334683024
short_window: 6, long_window: 11, total_weighted_pnl: 0.046945290055081114
short_window: 6, long_window: 12, total_weighted_pnl: 0.047726463769963166
short_window: 6, long_window: 13, total_weighted_pnl: 0.047979758586842426
short_window: 6, long_window: 14, total_weighted_pnl: 0.05005621326309836
Grid search finished.


In [5]:
grid_search_march_22_sorted

,short_window,long_window,total_weighted_pnl
11,6,14,0.050056
1,4,12,0.049469
3,4,14,0.049145
10,6,13,0.047980
9,6,12,0.047726
8,6,11,0.046945
0,4,11,0.046873
7,5,14,0.046793
6,5,13,0.044857
4,5,11,0.044585


# Run the trading strategy with the best EMA(Short & Long) params found by grid search

In [7]:
best_short_window = grid_search_march_22_sorted.iloc[0]['short_window']
best_long_window = grid_search_march_22_sorted.iloc[0]['long_window']
# best_short_window = 6
# best_long_window = 13
true_weight = 1

print('Best short window:', best_short_window)
print('Best long window:', best_long_window)
# Create a new DataFrame to store results with the best parameters
strategy_result = pd.DataFrame(columns=['ts_hour', 'pnl', 'EMA_short', 'EMA_long', 'is_positive', 'signal', 'count', 'consistency', 'is_continuous', 'class', 'weight_percentage', 'weight', 'weighted_pnl'])
results_list = []

# Update the DataFrame with the best parameters
for _, row in doge_df_march_22.iterrows():
    strategy_result = ud.update_dataframe_with_new_row(strategy_result, row, results_list, short_window=best_short_window, long_window=best_long_window, true_weight=true_weight)
print("Done")

Best short window: 6.0
Best long window: 14.0
Done


In [8]:
strategy_result[10:30].head(10)

,ts_hour,pnl,EMA_short,EMA_long,is_positive,signal,count,consistency,is_continuous,class,weight_percentage,weight,weighted_pnl
10,2022-03-01 10:00:00,-0.000724,0.000171,0.000039,0,0,0,NaN,NaN,0,0,0,0
11,2022-03-01 11:00:00,0.000162,0.000169,0.000055,1,0,0,NaN,NaN,0,0,0,0
12,2022-03-01 12:00:00,0.002809,0.000923,0.000422,1,0,0,NaN,NaN,0,0,0,0
13,2022-03-01 13:00:00,-0.002860,-0.000158,-0.000015,0,-1,0,NaN,0,0,0,0.0,-0.0
14,2022-03-01 14:00:00,0.000306,-0.000025,0.000028,1,0,0,NaN,0,2.3,48.75,0.4875,0.0
15,2022-03-01 15:00:00,0.004703,0.001326,0.000651,1,1,1,"[14, 15]",1,1,100,1.0,0.002293
16,2022-03-01 16:00:00,0.001261,0.001307,0.000732,1,0,-1,"[14, 15, 16]",1,2.2,48.75,0.4875,0.001261
17,2022-03-01 17:00:00,0.002944,0.001775,0.001027,1,0,0,NaN,0,2.3,48.75,0.4875,0.001435
18,2022-03-01 18:00:00,0.001619,0.001730,0.001106,1,0,0,NaN,0,2.3,48.75,0.4875,0.000789
19,2022-03-01 19:00:00,0.000443,0.001363,0.001018,1,0,0,NaN,0,2.3,48.75,0.4875,0.000216


In [9]:
strategy_result['class'].value_counts()

class
0      312
2.3    267
1      100
2.2     57
3.1      8
Name: count, dtype: int64

In [10]:
# import matplotlib.pyplot as plt
import plotly.graph_objects as go
import nbformat

from plotly.subplots import make_subplots

In [11]:
# Create the cumulative weighted PnL column
strategy_result_copy = strategy_result.copy()
strategy_result_copy['cumulative_weighted_pnl'] = strategy_result_copy['weighted_pnl'].cumsum()

In [12]:
# Create the plot
fig = go.Figure()

# Add the cumulative weighted PnL line
fig.add_trace(go.Scatter(
    x=strategy_result_copy['ts_hour'],
    y=strategy_result_copy['cumulative_weighted_pnl'],
    mode='lines',
    name='Cumulative Weighted PnL',
    line=dict(color='blue')
))

# Update layout
fig.update_layout(
    title='Cumulative Weighted PnL over Time with Annotations for Non-zero Classes',
    xaxis_title='Time',
    yaxis_title='Cumulative Weighted PnL',
    # template='plotly_white'
)


# Show the plot
fig.show()

In [13]:
# Define shape and color map for different classes
class_marker_map = {
    '1': {'color': 'red', 'symbol': 'circle', 'size': 10},
    '2.1': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '2.2': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '2.3': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '3.1': {'color': 'green', 'symbol': 'square', 'size': 6},
    '3.2': {'color': 'green', 'symbol': 'square', 'size': 6},
    '3.3': {'color': 'green', 'symbol': 'square', 'size': 6}
}

# Create the plot
fig = go.Figure()

# Add the cumulative weighted PnL line
fig.add_trace(go.Scatter(
    x=strategy_result_copy['ts_hour'],
    y=strategy_result_copy['cumulative_weighted_pnl'],
    mode='lines',
    name='Cumulative Weighted PnL',
    line=dict(color='blue')
))

# Add Class and Weight points to the plot
for class_type in strategy_result_copy['class'].unique():
    if class_type not in class_marker_map:
        continue
    class_df = strategy_result_copy[strategy_result_copy['class'] == class_type]
    marker = class_marker_map[str(class_type)]
    fig.add_trace(go.Scatter(
        x=class_df['ts_hour'],
        y=class_df['cumulative_weighted_pnl'],
        mode='markers',
        name=f'Class {class_type}',
        marker=dict(color=marker['color'], symbol=marker['symbol'], size=marker['size']),
        text=[f"Class: {row['class']}<br>Weight: {row['weight']:.2f}" for index, row in class_df.iterrows()],
        hoverinfo='text'
    ))

# Update layout
fig.update_layout(
    title='Cumulative Weighted PnL over Time with Annotations for Non-zero Classes',
    xaxis_title='Time',
    yaxis_title='Cumulative Weighted PnL',
    # template='plotly_white'
)

# Show the plot
fig.show()

In [16]:
strategy_result.weighted_pnl.sum()

1.0177950661992494